# Feature selection

In [8]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import plot_roc_curve
from IPython.display import clear_output
import random

In [31]:
df = pd.read_csv("../data/BTCUSDT-hour-target-features.csv")
results = pd.DataFrame(columns=["f1", "features_count", "weights", "mean", "features"])
lof = list(df.columns)[13:]
len(lof)

42

In [32]:
result = pd.Series([0, 0, ','.join(str(v) for v in list(np.full(len(lof), 100))), 0, ",".join([])], index=list(results.columns))
results = results.append(result, ignore_index=True)
results.to_csv("../data/features-weighted-results.csv", index=False)


In [27]:
results = pd.read_csv("../data/features-weighted-results.csv")

In [91]:
weights = list(map(int, list(results.weights.tail(1).replace("'", ''))[0].split(",")))

In [88]:
weights

'1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1'

In [19]:
a = pd.read_csv("../data/features-results.csv")

In [49]:
list(a.features.tail(1))[0].replace('[', '').replace(']', '').replace("'", '')

'volatility_atr_14, sma_20_diff_log_o_close_log, volume_eom_amplified, sma_20_o_close_log, volatility_ui_30, trend_macd_12_signal_log, volume_nvi_o_close_rate_50, sma_200_o_close_log, sma_20_diff_o_close, volume_mfi_50, volatility_bb_pband_100, trend_kst_sig_log, sma_20_o_close, volatility_dc_wband_10, sma_50_o_close_log, mom_tsi_20_log, sma_20_rate_log, sma_200_o_close, trend_dpo_100_log, mom_srsi_30, target'

In [113]:
int(results.tail(1)["mean"])

0

In [38]:
def find_weights(paths:str, metric:str, force_restart=False):
    """
    find weights
    """
    try:
        pd.read_csv(paths)
        if force_restart:
            raise ValueError('A very specific bad thing happened.')
    except:
        results = pd.DataFrame(columns=[metric, "features_count", "weights", "mean", "features"])
        result = pd.Series([0, 0, ','.join(str(v) for v in list(np.full(len(lof), 100))), 0, ",".join([])], index=list(results.columns))
        results = results.append(result, ignore_index=True)
        results.to_csv(paths, index=False)

    while True:
        results = pd.read_csv(paths)
        print(f"features tried: {len(results)}")

        weights = list(map(int, list(results.weights.tail(1).replace("'", ''))[0].split(",")))
        indicators = list(set(random.choices(population=lof, weights=weights, k=len(lof))))
        indicators.append("target")
    #     print(indicators)

        print(f"indicators: {indicators}")

        d = df.copy()
        d.dropna(inplace=True)

        d = d[list(indicators)].copy().astype(np.float32)

        X = d.drop(["target", "close"], axis=1, errors="ignore")[:30000]
        y = d["target"][:30000]

        model = RandomForestClassifier()
        p = np.mean(cross_val_score(model, X, y, cv=5, scoring=metric))
        prev_mean = np.mean(list(results.f1)[-int(0.2*len(results))-4:]) if len(results)>5 else p

        weights = change_weights_multiply(weights, p, prev_mean, indicators, lof) #Change weights

        mean = (prev_mean*5.+p)/(6.)
        result = pd.Series([p, len(indicators), ','.join(str(v) for v in list(weights)), mean, ",".join(indicators)], index=results.columns)
        results = results.append(result, ignore_index=True)
        results.to_csv(paths, index=False)
        clear_output()

In [39]:
def not_already_recorded(r:list[str]):
    """
    returns True if set of features are not already done before
    """
    if len(results["features"]) < 2:
        return True
#     result = list(r.replace("'", ''))[0].split(",")
    a = list(map(lambda x: set(list(x.replace("'", ''))[0].split(","))==set(r), list(results["features"][1:])))
    return True not in a

def change_weights(weights:list[int], precision:float, mean:float, indicators:list[str], lof:list[str]):
    """
    changes weight appropriately
    """
    amt = int(p*500-mean*500)
    weights = list(map(lambda f, w: w+amt if f in indicators else w, lof, weights))
    return list(map(lambda x: x if x>0 else 1, weights))

def change_weights_multiply(weights:list[int], precision:float, mean:float, indicators:list[str], lof:list[str]):
    """
    changes weight porportionally
    """
    amt = 3*(precision-mean) #0.01 or -0.015
    weights = list(map(lambda f, w: w+int(float(w)*amt) if f in indicators else w, lof, weights))
    return list(map(lambda x: x if x>0 else 1, weights))

In [ ]:
find_weights("../data/features-weighted-results-1.csv", "f1", True)

features tried: 858
indicators: ['volatility_wband_20', 'mom_srsi_10_d', 'volatility_bb_pband_100', 'bb', 'mom_srsi_15_k', 'volume_mfi_50', 'trend_sma_200_diff_o_close_log', 'volume_cmf_50_rate_200', 'mom_ppo_30_hist', 'volatility_bb_wband_200', 'volatility_dc_wband_10', 'volume_nvi_o_close_rate_50', 'target']
